# Example of an end to end ML project

Source: Hands-On Machine Learning witch Scikit-Learn, Keras & Tensorflow by Aurelien Geron

**Main steps**
1. Look at the big picture
2. Get the data
3. Discover and visualize the data to gain insights
4. Prepare data for ML algorithms
5. Select a model and train it
6. Fine-tune the model
7. Present solution
8. Lunch, monitor, and maintain your system

## Imports

In [1]:
# Standard libraries
import os
import tarfile
import urllib

# 3rd party libraries
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

%matplotlib inline

Hint Go thorugh a default check list. This works well for a lot of projects and gives you a starting piont that can be adpted to your needs.

# 1. Look at the big picture

#### 1.1 Frame the problem.
*Theory* <br>
A. What is the business objective? How does your organization aims to benefit from the mode? A good understanding here will help to frame the problem, algorithms to use, performance measurements, and prioritization. Ask what the current solution is, if there is any. This will give you a reference for performance and what to solve. <br> 
B. What system to use? Supervised, unsupervised, reinforcement learning, classification, etc. <br>

*Example* <br>
A. The model should predict if it is worth it to invest in a given area or not (price estimates / market price). Current solution is done manually, time cosuming and costly. The error is ca. 20%. <br>
B. Supervised learning task (given labled data-set), muliple univariant regression task (use of muliple features to predict a numeric value for each district), and plain batch learning (No data stream or need to rapidly adjust data and small enough to fit into memory).  

#### 1.2 Select performance measure
*Theory & Example* <br>
Typical performance measure for a regression system is the Root Mean Square Error. Good estimate for errors with high weight on large errors. 

#### 1.3 Check Assumptions
*Theory* <br>
Check assumptions to avoide mistakes early on. (Communication!)

*Example* <br>
We predict prices but we need to check if the downstream team will not actuall

# Glossary in order in which they appear during this chapter. 

Pipelines: Sequence of data processing components. They ususally work asynchronously and are connected via the data store. Here each component pulls data from a store, process is and stores the reults in another store.

Notation: <br>

m: number of instances in a dataset. <br>
x^i: feature vector. <br>
X: matrix containing all feature vectors of all dataset instances. <br>
h: sytems prediction function. <br>
RMSE(X,h): cost function. <br>